# **Librerias usadas**

In [68]:
import pandas as pd
import numpy as np
from meteostat import Stations, Daily
from datetime import datetime
import csv
import os
import requests
import re
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import folium
import random

# **Web scraping**

## **Sección 1: Inteligencia de red y extracción**

In [71]:
import requests
import re
import pandas as pd

def extraer_desde_html():
    print("🌐 Conectando con SENAMHI (Página Principal)...")
    
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, timeout=15)
        contenido = response.text
        
        # --- PASO 1: DIAGNÓSTICO VISUAL (INFALIBLE) ---
        # Buscamos la posición de la palabra "lat" y cortamos el texto alrededor.
        # Esto NO usa regex, así que no puede fallar al mostrarte los datos.
        indice = contenido.find('lat"') # Buscamos con comilla primero
        if indice == -1: 
            indice = contenido.find('lat') # Si falla, buscamos sin comilla
            
        print("\n" + "="*50)
        print("🔎 VISOR DE RAYOS X (Así se ven los datos reales):")
        print("="*50)
        # Mostramos 100 caracteres antes y después de encontrar "lat"
        print(contenido[indice-50 : indice+100])
        print("="*50 + "\n")

        # --- PASO 2: EXTRACCIÓN AUTOMÁTICA ---
        print("⛏️ Intentando extracción masiva...")
        
        # PATRÓN HÍBRIDO: 
        # Busca algo como: "nom": "LIMA", ... "lat": -12.04, "lon": -77.02
        # Funciona con o sin espacios extra.
        regex = r'"nom"\s*:\s*"(.*?)".*?"lat"\s*:\s*(-?\d+\.?\d*).*?"lon"\s*:\s*(-?\d+\.?\d*)'
        
        matches = re.findall(regex, contenido)
        
        if len(matches) > 0:
            print(f"✅ ¡ÉXITO! Se detectaron {len(matches)} estaciones.")
            
            datos = []
            for nombre, lat, lon in matches:
                datos.append({
                    "ESTACION": nombre.strip(),
                    "LATITUD": float(lat),
                    "LONGITUD": float(lon),
                    "FUENTE": "HTML_EMBEDDED"
                })
            
            # Guardar CSV
            df = pd.DataFrame(datos)
            df.to_csv("SENAMHI_ESTACIONES_FINAL.csv", index=False, encoding='utf-8-sig')
            print("📁 Archivo guardado: SENAMHI_ESTACIONES_FINAL.csv")
            print(df.head())
            
        else:
            print("⚠️ El patrón estándar falló. Revisa el texto en el 'VISOR DE RAYOS X' arriba.")
            print("Es posible que usen comillas simples (') en lugar de dobles (\") o que el orden sea diferente.")

    except Exception as e:
        print(f"❌ Error crítico: {e}")

if __name__ == "__main__":
    extraer_desde_html()

🌐 Conectando con SENAMHI (Página Principal)...

🔎 VISOR DE RAYOS X (Así se ven los datos reales):
= [
        {"nom": "MONTEGRANDE", "cate": "CO", "lat": -7.22499, "lon": -79.15323, "ico": "M", "cod": "100090","cod_old": "000396", "estado": "DIFERI

⛏️ Intentando extracción masiva...
✅ ¡ÉXITO! Se detectaron 968 estaciones.
📁 Archivo guardado: SENAMHI_ESTACIONES_FINAL.csv
              ESTACION  LATITUD  LONGITUD         FUENTE
0          MONTEGRANDE -7.22499 -79.15323  HTML_EMBEDDED
1           YONAN GORE -7.25592 -79.09908  HTML_EMBEDDED
2         QUEROCOTILLO -6.27366 -79.03696  HTML_EMBEDDED
3  PUENTE  MAYGASBAMBA -6.67411 -78.52437  HTML_EMBEDDED
4      QUEBRADA SHUGAR -6.68778 -78.45694  HTML_EMBEDDED


## **Sección 2: El buscador de vecinos**

In [73]:
def encontrar_estacion_cercana(lat_objetivo, lon_objetivo, archivo_csv="SENAMHI_ESTACIONES_FINAL.csv"):
    """
    Toma una coordenada (de Visual Crossing/OpenSky) y encuentra la estación SENAMHI más cercana.
    """
    # 1. Cargar la base de datos que acabamos de crear
    try:
        df = pd.read_csv(archivo_csv)
    except FileNotFoundError:
        return "Error: No se encuentra el archivo CSV de estaciones."

    # 2. Calcular la distancia a CADA estación (Fórmula de distancia simple)
    # Distancia = Raíz((Lat1-Lat2)² + (Lon1-Lon2)²)
    df['distancia'] = np.sqrt(
        (df['LATITUD'] - lat_objetivo)**2 + 
        (df['LONGITUD'] - lon_objetivo)**2
    )

    # 3. Ordenar y tomar la ganadora (la de menor distancia)
    estacion_ganadora = df.sort_values('distancia').iloc[0]

    return {
        "ESTACION": estacion_ganadora['ESTACION'],
        "LATITUD": estacion_ganadora['LATITUD'],
        "LONGITUD": estacion_ganadora['LONGITUD'],
        "DISTANCIA_APROX": round(estacion_ganadora['distancia'] * 111, 2) # *111 para convertir grados a km aprox
    }

# --- PRUEBA DEL FLUJO (SIMULACIÓN) ---
print("--- 🔄 SIMULANDO PASE DE BATUTA ENTRE ETAPAS ---")

# 1. Output de ETAPA 1 (Visual Crossing) para "Lima"
lat_vc = -12.02
lon_vc = -77.11
print(f"📍 Dato recibido de Etapa 1: {lat_vc}, {lon_vc}")

# 2. Ejecución de ETAPA 3 (Búsqueda SENAMHI)
resultado = encontrar_estacion_cercana(lat_vc, lon_vc)

print("\n🎯 RESULTADO ETAPA 3:")
print(f"   La estación SENAMHI más cercana es: {resultado['ESTACION']}")
print(f"   Ubicada a {resultado['DISTANCIA_APROX']} km del punto de vuelo.")
print(f"   Coordenadas reales: {resultado['LATITUD']}, {resultado['LONGITUD']}")

--- 🔄 SIMULANDO PASE DE BATUTA ENTRE ETAPAS ---
📍 Dato recibido de Etapa 1: -12.02, -77.11

🎯 RESULTADO ETAPA 3:
   La estación SENAMHI más cercana es: SAN MARTIN DE PORRES
   Ubicada a 3.09 km del punto de vuelo.
   Coordenadas reales: -12.00889, -77.08447


## **Sección 3: Correlación espacial**

In [75]:
def validar_cobertura_espacial(datos_estacion):
    """
    SECCIÓN 3: AUDITORÍA DE CORRELACIÓN
    -----------------------------------------------------
    Input: Diccionario con la estación y su distancia (Output de Sección 2).
    Proceso: Evalúa si la estación está dentro de un radio útil.
    Output: Calificación de confiabilidad (Semáforo) y recomendación.
    """
    if not datos_estacion:
        return {"ESTADO": "ERROR", "MENSAJE": "No hay datos de estación para auditar."}

    distancia = datos_estacion['DISTANCIA_KM']
    estacion = datos_estacion['ESTACION']

    # --- REGLAS DE NEGOCIO (Umbrales de Distancia) ---
    # Ajustables según la sensibilidad de tu proyecto (Drones vs Aviones)
    
    if distancia <= 5.0:
        # Rango Óptimo (Barrios vecinos)
        return {
            "NIVEL_CONFIANZA": "🟢 ALTA (Excelente)",
            "REPRESENTATIVIDAD": "100% - Datos locales precisos",
            "ACCION": f"✅ Usar datos de '{estacion}' sin restricciones.",
            "RIESGO_CLIMATICO": "Bajo (Microclima idéntico)"
        }
    
    elif 5.0 < distancia <= 20.0:
        # Rango Aceptable (Misma ciudad/valle)
        return {
            "NIVEL_CONFIANZA": "🟡 MEDIA (Aceptable)",
            "REPRESENTATIVIDAD": "70% - Variación posible",
            "ACCION": f"⚠️ Usar datos de '{estacion}' con margen de error.",
            "RIESGO_CLIMATICO": "Medio (Posible variación de viento/lluvia)"
        }
    
    else:
        # Rango Crítico (Demasiado lejos)
        return {
            "NIVEL_CONFIANZA": "🔴 BAJA (No Confiable)",
            "REPRESENTATIVIDAD": "< 30% - Clima probablemente diferente",
            "ACCION": f"❌ DESCARTAR. '{estacion}' está muy lejos para validar.",
            "RIESGO_CLIMATICO": "Alto (No refleja la realidad del punto)"
        }

# ==========================================
# INTEGREMOS TODO (Simulación Final)
# ==========================================
if __name__ == "__main__":
    # 1. Simulación de lo que obtuviste en la Sección 2 (San Martín de Porres)
    resultado_seccion_2 = {
        "ESTACION": "SAN MARTIN DE PORRES",
        "LATITUD": -12.00889,
        "LONGITUD": -77.08447,
        "DISTANCIA_KM": 3.09,    # <--- Este dato es clave
        "FUENTE": "SENAMHI_MATCH"
    }

    print(f"📡 Estación candidata: {resultado_seccion_2['ESTACION']} ({resultado_seccion_2['DISTANCIA_KM']} km)")
    
    # 2. Ejecutar Sección 3
    auditoria = validar_cobertura_espacial(resultado_seccion_2)
    
    # 3. Reporte
    print("\n📋 REPORTE DE AUDITORÍA ESPACIAL:")
    print("-" * 40)
    print(f"Confianza:      {auditoria['NIVEL_CONFIANZA']}")
    print(f"Análisis:       {auditoria['REPRESENTATIVIDAD']}")
    print(f"Recomendación:  {auditoria['ACCION']}")
    print("-" * 40)

📡 Estación candidata: SAN MARTIN DE PORRES (3.09 km)

📋 REPORTE DE AUDITORÍA ESPACIAL:
----------------------------------------
Confianza:      🟢 ALTA (Excelente)
Análisis:       100% - Datos locales precisos
Recomendación:  ✅ Usar datos de 'SAN MARTIN DE PORRES' sin restricciones.
----------------------------------------


## **Sección 4: Detección de clima**

In [77]:
# ==========================================
# PARTE A: EL BUSCADOR (Necesario para que funcione el flujo)
# ==========================================
def encontrar_estacion_cercana(lat_objetivo, lon_objetivo, archivo_csv="SENAMHI_ESTACIONES_FINAL.csv"):
    """
    Busca la estación más cercana leyendo el CSV generado previamente.
    """
    try:
        df = pd.read_csv(archivo_csv)
    except FileNotFoundError:
        print("❌ Error: No se encuentra 'SENAMHI_ESTACIONES_FINAL.csv'. Ejecuta primero la extracción.")
        return None

    # Cálculo de distancia
    df['distancia'] = np.sqrt(
        (df['LATITUD'] - lat_objetivo)**2 + 
        (df['LONGITUD'] - lon_objetivo)**2
    )

    # Ordenar y ganar
    estacion_ganadora = df.sort_values('distancia').iloc[0]

    return {
        "ESTACION": estacion_ganadora['ESTACION'],
        "LATITUD": estacion_ganadora['LATITUD'],
        "LONGITUD": estacion_ganadora['LONGITUD'],
        "DISTANCIA_KM": round(estacion_ganadora['distancia'] * 111, 2)
    }

# ==========================================
# PARTE B: EL SENSOR (Simulación de clima)
# ==========================================
def obtener_datos_sensor(estacion_info):
    """
    Simula la lectura de humedad y neblina basada en la estación encontrada.
    """
    if not estacion_info:
        return None
        
    nombre = estacion_info['ESTACION']
    distancia = estacion_info['DISTANCIA_KM']
    
    print(f"\n🌡️ CONECTANDO CON SENSOR DE: {nombre}...")
    
    # 1. Validación de distancia
    if distancia > 10:
        return {
            "TIENE_NEBLINA": False,
            "HUMEDAD": "Desconocida",
            "CONDICION": "⚠️ Fuera de rango",
            "MENSAJE": "⚠️ Estación demasiado lejos."
        }
    
    # 2. Simulación del sensor (Humedad alta por ser costa)
    humedad_actual = random.randint(85, 99) 
    
    tiene_neblina = False
    condicion = ""
    
    if humedad_actual > 90:
        tiene_neblina = True
        condicion = "🌫️ NEBLINA DENSA"
    elif humedad_actual > 80:
        condicion = "💧 ALTA HUMEDAD"
    else:
        condicion = "☀️ DESPEJADO"

    return {
        "TIENE_NEBLINA": tiene_neblina,
        "HUMEDAD": f"{humedad_actual}%",
        "CONDICION": condicion,
        "MENSAJE": "✅ Lectura exitosa."
    }

# ==========================================
# EJECUCIÓN FINAL
# ==========================================
if __name__ == "__main__":
    # 1. INPUT (Tus coordenadas)
    lat = -12.02
    lon = -77.11
    
    print(f"📍 Coordenadas recibidas: {lat}, {lon}")

    # 2. BUSCAR ESTACIÓN
    estacion = encontrar_estacion_cercana(lat, lon) 
    
    if estacion:
        print(f"✅ Estación base encontrada: {estacion['ESTACION']} ({estacion['DISTANCIA_KM']} km)")
        
        # 3. LEER SENSOR
        clima = obtener_datos_sensor(estacion)
        
        print("\n" + "="*40)
        print("🤖 REPORTE FINAL DE VALIDACIÓN (ETAPA 3)")
        print("="*40)
        print(f"💧 Humedad:       {clima['HUMEDAD']}")
        print(f"👀 Visibilidad:   {clima['CONDICION']}")
        print(f"🚦 ¿HAY NEBLINA?: {'SÍ' if clima['TIENE_NEBLINA'] else 'NO'}")
        print("="*40)
    else:
        print("❌ No se pudo encontrar una estación cercana.")

📍 Coordenadas recibidas: -12.02, -77.11
✅ Estación base encontrada: SAN MARTIN DE PORRES (3.09 km)

🌡️ CONECTANDO CON SENSOR DE: SAN MARTIN DE PORRES...

🤖 REPORTE FINAL DE VALIDACIÓN (ETAPA 3)
💧 Humedad:       89%
👀 Visibilidad:   💧 ALTA HUMEDAD
🚦 ¿HAY NEBLINA?: NO


## **Sección 5: Resultados**

In [79]:
def graficar_auditoria_espacial(lat_objetivo, lon_objetivo, datos_estacion):
    """
    SECCIÓN 4: RESULTADOS VISUALES
    -----------------------------------------------------
    Genera un mapa interactivo (.html) que muestra la relación espacial
    entre el punto de interés y la estación meteorológica seleccionada.
    """
    if not datos_estacion:
        print("❌ No hay datos para graficar.")
        return

    print("🎨 Generando mapa de validación espacial...")

    # 1. Crear el mapa base centrado entre los dos puntos
    centro_lat = (lat_objetivo + datos_estacion['LATITUD']) / 2
    centro_lon = (lon_objetivo + datos_estacion['LONGITUD']) / 2
    
    m = folium.Map(location=[centro_lat, centro_lon], zoom_start=13)

    # 2. MARCADOR A: Tu Punto Objetivo (Ej. Dron/Avión)
    folium.Marker(
        location=[lat_objetivo, lon_objetivo],
        popup="📍 TU PUNTO OBJETIVO (Visual Crossing/OpenSky)",
        icon=folium.Icon(color="red", icon="plane", prefix="fa")
    ).add_to(m)

    # 3. MARCADOR B: Estación SENAMHI
    texto_estacion = f"📡 {datos_estacion['ESTACION']} ({datos_estacion['DISTANCIA_KM']} km)"
    folium.Marker(
        location=[datos_estacion['LATITUD'], datos_estacion['LONGITUD']],
        popup=texto_estacion,
        icon=folium.Icon(color="green", icon="cloud", prefix="fa")
    ).add_to(m)

    # 4. LÍNEA DE CONEXIÓN (El vector de distancia)
    folium.PolyLine(
        locations=[
            [lat_objetivo, lon_objetivo], 
            [datos_estacion['LATITUD'], datos_estacion['LONGITUD']]
        ],
        color="blue",
        weight=2,
        opacity=0.7,
        dash_array='5, 10', # Línea punteada
        tooltip=f"Distancia: {datos_estacion['DISTANCIA_KM']} km"
    ).add_to(m)

    # 5. RADIO DE CONFIANZA (Auditoría visual)
    # Dibujamos un círculo de 5km (5000 metros) alrededor de la estación
    # Si tu punto rojo cae dentro del círculo, la validación es "ALTA"
    folium.Circle(
        location=[datos_estacion['LATITUD'], datos_estacion['LONGITUD']],
        radius=5000, # 5000 metros = 5km
        color="green",
        fill=True,
        fill_opacity=0.1,
        popup="✅ Zona de Alta Confiabilidad (5km)"
    ).add_to(m)

    # 6. Guardar el resultado
    nombre_archivo = "MAPA_VALIDACION_RESULTADOS.html"
    m.save(nombre_archivo)
    print(f"✅ ¡Mapa generado! Abre el archivo '{nombre_archivo}' en tu navegador.")
    
    return m # Retorna el mapa por si estás en Jupyter Notebook para verlo ahí mismo

# ==========================================
# EJECUCIÓN FINAL (INTEGRACIÓN)
# ==========================================
if __name__ == "__main__":
    # Simulamos que ya tenemos los datos de la Sección 2
    # (En tu código real, esto viene de la variable 'mejor_estacion')
    lat_input = -12.02
    lon_input = -77.11
    
    resultado_mock = {
        "ESTACION": "SAN MARTIN DE PORRES",
        "LATITUD": -12.00889,
        "LONGITUD": -77.08447,
        "DISTANCIA_KM": 3.09
    }

    # Llamamos a la gráfica
    graficar_auditoria_espacial(lat_input, lon_input, resultado_mock)

🎨 Generando mapa de validación espacial...
✅ ¡Mapa generado! Abre el archivo 'MAPA_VALIDACION_RESULTADOS.html' en tu navegador.


# **¿Qué tiempo hace en esas estaciones y qué tan peligroso es para volar?**

# **Motor de búsqueda y enriquecimiento operativo**

## **Análisis exploratorio de datos**

Objetivo:
- Mostrar el DataFrame.head()
- Explicar qué variables se extrajeron
- info()
- describe()

In [84]:
# ===============================
# 1. OBTENER ESTACIONES EN PERÚ
# ===============================
def obtener_estaciones_peru():
    estaciones = Stations()
    estaciones = estaciones.region('PE')
    df_estaciones = estaciones.fetch()
    return df_estaciones


# ===============================
# 2. OBTENER DATOS CLIMÁTICOS
# ===============================
def obtener_datos_climaticos(lat, lon, inicio, fin):
    estaciones = Stations().nearby(lat, lon)
    estacion = estaciones.fetch(1)

    if estacion.empty:
        return pd.DataFrame()

    data = Daily(estacion.index[0], inicio, fin)
    df = data.fetch()
    return df


# ===============================
# 3. EJECUCIÓN PRINCIPAL
# ===============================
if __name__ == "__main__":

    # --- Paso 1: Estaciones ---
    print(" Obteniendo estaciones en Perú...")
    estaciones_pe = obtener_estaciones_peru()

    print("\n Vista previa de estaciones:")
    print(estaciones_pe.head())

    # --- Paso 2: Datos climáticos (ejemplo Lima) ---
    lat_lima = -12.0464
    lon_lima = -77.0428

    inicio = datetime(2024, 1, 1)
    fin = datetime(2024, 12, 31)

    print("\n Descargando datos climáticos para Lima...")
    df_clima = obtener_datos_climaticos(lat_lima, lon_lima, inicio, fin)

    # --- Paso 3: Guardar CSV ---
    nombre_csv = "clima_lima_2024.csv"
    df_clima.to_csv(nombre_csv)
    print(f"\n Archivo CSV generado: {nombre_csv}")

    # --- Paso 4: Mostrar DataFrame ---
    print("\n Vista previa del DataFrame climático:")
    print(df_clima.head())

    print("\n Información del DataFrame:")
    print(df_clima.info())

    print("\n Estadísticas descriptivas:")
    print(df_clima.describe())


 Obteniendo estaciones en Perú...

 Vista previa de estaciones:
              name country region    wmo  icao  latitude  longitude  \
id                                                                    
84370       Tumbes      PE     TU  84370  SPME   -3.5500     -80.40   
84377      Iquitos      PE     LO  84377  SPQT   -3.7500     -73.25   
84390       Talara      PE     PI  84390  SPYL   -4.5667     -81.25   
84401        Piura      PE     PI  84401  SPUR   -5.1833     -80.60   
84405  Huancabamba      PE     PI  84405  SPAB   -5.2333     -79.45   

       elevation      timezone hourly_start hourly_end daily_start  daily_end  \
id                                                                              
84370       25.0  America/Lima   1974-03-15 2025-12-15  1974-03-15 2025-05-27   
84377      125.0  America/Lima   1973-01-01 2025-12-15  1973-01-01 2025-06-24   
84390       85.0  America/Lima   1933-02-09 2025-12-14  1942-11-01 2025-08-24   
84401       49.0  America/Lima   

Conclusión:
- El análisis exploratorio permitió validar la estructura y calidad del conjunto de datos, identificando variables relevantes y descartando aquellas con información incompleta, asegurando así una base adecuada para análisis posteriores.

## **Caracterización Climática y Generación de Indicadores Meteorológicos**

Objetivo: 
- Transformar los datos climáticos crudos del SENAMHI en indicadores meteorológicos operativos, listos para ser usados más adelante por el pipeline aéreo.

### **Selección de variables meteorológicas relevantes**

| Variable | Uso operativo                    |
| -------- | -------------------------------- |
| `wspd`   | Impacto en despegue y aterrizaje |
| `prcp`   | Riesgo de pista mojada           |
| `pres`   | Estabilidad atmosférica          |
| `tmax`   | Densidad del aire                |
| `tmin`   | Condiciones nocturnas            |

- Se seleccionaron variables meteorológicas con impacto directo en la operación aérea, descartando aquellas sin registros completos.

### **Limpieza y tratamiento de valores faltantes**

Objetivo:
- Garantizar la calidad y consistencia del conjunto de datos climático antes de su transformación en indicadores operativos.

### **Construcción de indicadores climáticos**

Objetivo:
- Transformar las variables meteorológicas continuas en indicadores que permitan detectar condiciones climáticas potencialmente adversas.
- A partir de las variables climáticas seleccionadas se construyeron indicadores binarios que permiten identificar eventos meteorológicos relevantes para la operación aérea.

Indicadores definidos:
- Viento fuerte
- Precipitación intensa
- Presión atmosférica baja

### **Clasificación de condiciones meteorológicas**

Objetivo:
- Etiquetar cada observación temporal según el nivel de severidad climática, facilitando su integración futura con datos operativos.
- Finalmente, se clasificaron las condiciones meteorológicas en categorías de severidad climática, permitiendo una interpretación directa del impacto potencial del clima.

Categorías:
- Normal
- Viento adverso
- Precipitación intensa
- Condición severa

## **Código-SENAMHI**

### **Extracción de datos**

In [101]:
def obtener_datos_climaticos(lat, lon, inicio, fin):
    est = Stations().nearby(lat, lon).fetch(1)
    if est.empty:
        return pd.DataFrame()
    return Daily(est.index[0], inicio, fin).fetch()

# --- EJECUCIÓN ---
df = obtener_datos_climaticos(
    -12.0464, -77.0428,
    datetime(2024, 1, 1),
    datetime(2024, 12, 31))

print(df)

            tavg  tmin  tmax  prcp  snow  wdir  wspd  wpgt    pres  tsun
time                                                                    
2024-01-01  23.4  22.0  25.2   1.0  <NA>  <NA>  19.1  <NA>  1014.0  <NA>
2024-01-02  23.8  22.0  26.0   0.0  <NA>  <NA>  20.9  <NA>  1013.4  <NA>
2024-01-03  23.5  22.0  25.3   0.0  <NA>  <NA>  21.5  <NA>  1013.5  <NA>
2024-01-04  23.6  21.8  26.0   0.0  <NA>  <NA>  16.4  <NA>  1014.1  <NA>
2024-01-05  23.9  22.0  26.0   0.0  <NA>  <NA>  24.7  <NA>  1013.2  <NA>
...          ...   ...   ...   ...   ...   ...   ...   ...     ...   ...
2024-12-27  22.5  21.0  26.0   0.0  <NA>  <NA>  18.6  <NA>  1013.9  <NA>
2024-12-28  21.1  19.0  23.0   0.0  <NA>  <NA>  18.9  <NA>  1013.0  <NA>
2024-12-29  20.9  19.0  23.0   0.0  <NA>  <NA>  19.3  <NA>  1012.9  <NA>
2024-12-30  21.4  20.0  23.2   0.0  <NA>  <NA>  16.8  <NA>  1012.7  <NA>
2024-12-31  21.2  19.0  24.0   0.0  <NA>  <NA>  13.1  <NA>  1012.6  <NA>

[366 rows x 10 columns]


### **Limpieza de datos**

In [103]:
def limpiar_datos_climaticos(df):
    columnas_utiles = ["tavg", "tmin", "tmax", "prcp", "wspd", "pres"]
    df_limpio = df[columnas_utiles].copy()
    return df_limpio

### **Construcción de indicadores**

In [105]:
def construir_indicadores(df):
    df_ind = df.copy()

    # Rellenar NA SOLO para indicadores
    df_ind["prcp"] = df_ind["prcp"].fillna(0)
    df_ind["wspd"] = df_ind["wspd"].fillna(df_ind["wspd"].median())
    df_ind["pres"] = df_ind["pres"].fillna(df_ind["pres"].median())

    # Umbrales
    viento_umbral = df_ind["wspd"].quantile(0.90)
    lluvia_umbral = 5.0
    presion_umbral = df_ind["pres"].quantile(0.10)

    df_ind["viento_fuerte"] = (df_ind["wspd"] > viento_umbral).astype(int)
    df_ind["lluvia_intensa"] = (df_ind["prcp"] > lluvia_umbral).astype(int)
    df_ind["presion_baja"] = (df_ind["pres"] < presion_umbral).astype(int)

    return df_ind

### **Clasificación climática**

In [107]:
def clasificar_condicion(df):
    df = df.copy()
    
    score = (
        df["viento_fuerte"] +
        df["lluvia_intensa"] +
        df["presion_baja"])

    df["condicion_climatica"] = np.select(
        [score == 0, score == 1, score >= 2],
        ["Normal", "Condición moderada", "Condición severa"])
    
    return df

### **Ejecución principal + Clasificación**

In [109]:
if __name__ == "__main__":

    lat_lima = -12.0464
    lon_lima = -77.0428

    inicio = datetime(2024, 1, 1)
    fin = datetime(2024, 12, 31)

    df_clima = obtener_datos_climaticos(lat_lima, lon_lima, inicio, fin)
    df_limpio = limpiar_datos_climaticos(df_clima)
    df_indicadores = construir_indicadores(df_limpio)
    df_final = clasificar_condicion(df_indicadores)

    # Guardar CSV
    df_final.to_csv("senamhi_clima_indicadores.csv")

    # VISUALIZACIÓN EN PANDAS (como pediste)
    print("\n Vista previa del dataset final:")
    print(df_final.head())

    print("\n Resumen del dataset:")
    print(df_final.info())


 Vista previa del dataset final:
            tavg  tmin  tmax  prcp  wspd    pres  viento_fuerte  \
time                                                              
2024-01-01  23.4  22.0  25.2   1.0  19.1  1014.0              0   
2024-01-02  23.8  22.0  26.0   0.0  20.9  1013.4              1   
2024-01-03  23.5  22.0  25.3   0.0  21.5  1013.5              1   
2024-01-04  23.6  21.8  26.0   0.0  16.4  1014.1              0   
2024-01-05  23.9  22.0  26.0   0.0  24.7  1013.2              1   

            lluvia_intensa  presion_baja condicion_climatica  
time                                                          
2024-01-01               0             0              Normal  
2024-01-02               0             0  Condición moderada  
2024-01-03               0             0  Condición moderada  
2024-01-04               0             0              Normal  
2024-01-05               0             0  Condición moderada  

 Resumen del dataset:
<class 'pandas.core.frame.DataFr